In [50]:
#Submission1:
#Predicting Floor, Latitude, Longitude on TRAIN

In [51]:
#Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import math
import scipy

#Import Models
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

#Model Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

#Cross Validation
from sklearn.model_selection import train_test_split
from math import sqrt

In [52]:
#Import Datasets
wifi = pd.read_csv("./trainingData.csv")
wifi_test = pd.read_csv("./testData.csv")

wifi_test.shape

(5179, 529)

In [53]:
#### DELETE COLUMNS #####

del wifi['RELATIVEPOSITION']
del wifi['USERID']
del wifi['TIMESTAMP']
del wifi_test['RELATIVEPOSITION']
del wifi_test['USERID']
del wifi_test['TIMESTAMP']

#### DUPLICATED ROWS #####

wifi = wifi.drop_duplicates(subset=None, keep='first',inplace=False) 


#### STRANGE AND LOW SIGNALS #####

wifi.iloc[:, 0:520] = wifi.iloc[:, 0:520].replace(np.r_[1:9], 10) 
wifi_test.iloc[:, 0:520] = wifi_test.iloc[:, 0:520].replace(np.r_[1:9], 10) 
#Turn useless signals to a single useless number (10)

wifi.iloc[:, 0:520] = wifi.iloc[:, 0:520].replace(np.r_[71:106], 70)
wifi_test.iloc[:, 0:520] = wifi_test.iloc[:, 0:520].replace(np.r_[71:106], 70)
#Reduce perfect signals above 70 to 70.


In [54]:
#BUILDING
x_train = wifi.iloc[:,0:520]
y_train = wifi.iloc[:,np.r_[520:524]]
x_test = wifi_test.iloc[:,0:520]
y_test = wifi_test.iloc[:,np.r_[520:524]]

LR_B = LogisticRegression(
    random_state=0, solver='lbfgs', multi_class='multinomial').fit(x_train, y_train['BUILDINGID'])

#Predictions
LRpredictions_building = LR_B.predict(x_test)

/home/s/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning:

lbfgs failed to converge. Increase the number of iterations.



In [55]:
#FLOOR
x_train['BUILDINGID'] = y_train['BUILDINGID']
x_test['BUILDINGID'] = LRpredictions_building

KNN_F = KNeighborsClassifier(n_neighbors=1).fit(x_train, y_train['FLOOR'])

#Predictions
KNNpredictions_floor = KNN_F.predict(x_test)

/home/s/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [56]:
#LATITUDE
x_train['FLOOR'] = y_train['FLOOR']
x_test['FLOOR'] = KNNpredictions_floor

KNN_LAT = KNeighborsRegressor(n_neighbors=4).fit(x_train, y_train['LATITUDE'])

#Predictions
KNNpredictions_latitude = KNN_LAT.predict(x_test)

/home/s/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [57]:
#LONGITUDE
x_train['LATITUDE'] = y_train['LATITUDE']
x_test['LATITUDE'] = KNNpredictions_latitude

KNN_LONG = KNeighborsRegressor(n_neighbors=1).fit(x_train,y_train['LONGITUDE'])

#Predictions
KNNpredictions_longitude = KNN_LONG.predict(x_test)

/home/s/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [58]:
KNNpredictions_latitude = pd.DataFrame(KNNpredictions_latitude)
KNNpredictions_longitude = pd.DataFrame(KNNpredictions_longitude)
KNNpredictions_floor = pd.DataFrame(KNNpredictions_floor)

submission1 = pd.concat([KNNpredictions_latitude,KNNpredictions_longitude,KNNpredictions_floor],axis=1)
submission1.shape

(5179, 3)

In [59]:
submission1.columns = ["LATITUDE","LONGITUDE","FLOOR"]
submission1.to_csv("./submission1",index = False)